# CFT2018 contest submit

In [1]:
import pandas as pd
import numpy as np

# pip install symspell - Для коррекции опечаток 
import symspellpy

import os
import gc
import pickle

In [2]:
PATH_TO_DATA = ('D:/Py/DataFrames/CFT_Contest(Datasouls)/')

Load data and best solo model predictions 

In [3]:
train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'))
test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'))

test_oof = pd.read_csv(os.path.join(PATH_TO_DATA, 'predictions/logreg_and_binary_preds_test.csv'))

In [6]:
test['target'] = np.argmax(test_oof.values, axis=1)

In [7]:
test.head()

,id,fullname,country,target
0,0,ХУДАШКУРОВА ГУЛЗХОДА БЕРДИЕВНА,УЗБЕКИСТАН,1
1,1,СВЕЖЕТЬФЛОГИСТОН АРСЕН,РОССИЯ,2
2,2,ГУЛОМОВА СОЖИДА САНАЕВНА,УЗБЕКИСТАН,0
3,3,КАМПЫШЕВА ГУЛЯИМ БЕЙСЕМБАЕВНА,КАЗАХСТАН,0
4,4,OROSUMEBTOV MIRLAN,РОССИЯ,0


### Typos correction

To correct typos, use the open-source library ([github](https://github.com/mammothb/symspellpy)).

In [14]:
symspell = symspellpy.SymSpell()

Prepare training sample (use only train correct full names!)

In [15]:
train.loc[train.target != 1, 'fullname_true'] = train.loc[train.target != 1, 'fullname'] 
dicts = [name for person in train.fullname_true for name in person.split(' ')]

from collections import Counter
name_freq = Counter(dicts)

In [16]:
with open('dictionary.txt', 'w') as f:
    for name, freq in name_freq.items():
        f.write('{} {}\n'.format(name, freq))

In [17]:
!head dictionary.txt

AKHMEDOV 381
YURIY 98
ФОЗИЛОВ 783
РАМИЛЬ 2383
ГУЛОВИЧ 190
ГОИБОВ 482
АХЛИДДИН 753
ШАМСУДИНОВИЧ 183
ХУСНУТДИНОВА 42
МАРГАРИТА 1923


Load the dictionary and check accuracy on train data (overfitted!)

In [18]:
symspell.load_dictionary('dictionary.txt', term_index=0, count_index=1)

True

In [19]:
def correct(s):
    def correct_word(w):
        tmp = symspell.lookup(w, symspellpy.Verbosity.CLOSEST)
        if len(tmp):
            return tmp[0].term.upper()
        else:
            return w

    return ' '.join([correct_word(word) for word in s.split(' ')])

In [20]:
%%time
correct('КАРАБОЗОВа ЛАТИФ АЛИМАМАДОВИЧ')

CPU times: user 1.85 ms, sys: 74 µs, total: 1.93 ms
Wall time: 1.97 ms


'КАРАБОЗОВ ЛАТИФ АЛИМАМАДОВИЧ'

In [21]:
train_1 = train.loc[train.target == 1].copy()
train_1['fullname_corrected'] = train_1.fullname.apply(correct)

In [22]:
np.mean(train_1.fullname_true == train_1.fullname_corrected)

0.8272597104120801

Correct test samples and save our submission

In [23]:
test['fullname_true'] = None
test.loc[test.target == 1, 'fullname_true'] = test.loc[test.target == 1, 'fullname'].apply(correct)

In [24]:
test[['id', 'target', 'fullname_true']].to_csv('final_submission.csv', index=False)